# Examining The Influence Of Square Footage On The Selling Price Of Houses In Vancouver #

## Introduction: ##
The price of a house can depend on a multitude of factors, including location, size, the age of the home, and many others. Based on information about the property, predictions can be made about the selling price of a home compared to others in the area.

The goal of our project is to utilize regression analysis to determine the relationship between price and total square footage of houses in Vancouver. We will do this using a  [publicly available dataset from the website Kaggle](https://www.kaggle.com/datasets/darianghorbanian/vancouver-home-price-analysis-regression), which has price data, square footage, and other details about Vancouver houses from 2017-2020. 

Our Predictive Question is: “Does the square footage of a house in Vancouver have an impact on its selling price?”